[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [12]:
# Import pandas, numpy
import numpy as np
import pandas as pd

In [13]:
# Read dataset with extra params sep='\t', encoding="latin-1"
data = pd.read_csv("/content/labeledTrainData.tsv", sep='\t', encoding="latin-1")
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [14]:
# stop words
from nltk.corpus import stopwords

In [15]:
# Removing special characters and "trash"
import re

def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [16]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
lemmatizer = PorterStemmer()

def tokenizer_porter(text):
    return[lemmatizer.stem(word) for word in text.split()]

In [17]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
y = data.sentiment.values
X = data.review.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [18]:
# Import LogisticRegression, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer(strip_accents=None, lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer_porter, use_idf=True, smooth_idf=True)
X = tfidf.fit_transform(data.review)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5, shuffle=False)
clf = Pipeline([('keyphase_vect', tfidf), ('clf', LogisticRegression())])
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('keyphase_vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7fcef1e70400>,
                                 smooth_idf=True, stop_words=None,
                                 strip_accen...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7fcef1fd3d08>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
            

## 4. Evaluate Model

In [21]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import classification_report
print(clf.score(X_test, y_test))
predict = clf.predict(X_test)
classification_report(y_test, predict, target_names=['Positive','Negative'])


0.85084940973222


'              precision    recall  f1-score   support\n\n    Positive       0.85      0.84      0.85      1691\n    Negative       0.85      0.86      0.86      1782\n\n    accuracy                           0.85      3473\n   macro avg       0.85      0.85      0.85      3473\nweighted avg       0.85      0.85      0.85      3473\n'

## 5. Export Model 

In [22]:
# Using pickle to export our trained model
import pickle
import os
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))